# GEOPHYS 257 (Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

## Multicore Parallelization of Matrix Multiplication Using Numba

In this lab we will be using Numba to accelerate matrix-matrix multiplications by exploiting parallelism. Even most laptops today have Multicore CPUs, where a *core* is a microprocessor, and each core is usually a copy of the each other core. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba](https://numba.readthedocs.io/en/stable/index.html): Documentation
* [Numba in 30 min](https://youtu.be/DPyPmoeUdcE): Conference presentation video


## Required Preperation
Please watch the following videos before starting the lab (each is pretty short):
* [Introduction to Parallel Computing](https://youtu.be/RNVIcm8-6RE)
* [Amdahl's Law](https://youtu.be/Axx2xuB-Xuo)
* [CPU Caching](https://youtu.be/KmairurdiaY)
* [Pipelining](https://youtu.be/zPmfprtdzCE)
* [Instruction Level Parallelism](https://youtu.be/ZoDUI3gTkDI)
* [Introduction to SIMD](https://youtu.be/o_n4AKwdfiA)


### Exercise 0

Please run the following cells.  Examine the result of the example function that makes use of the *timer* wrapper. Also please use this timer wrapper (defined below) for all the exercises that follow.

#### Load python modules (Note: you may need to install some Python packages for the modules below, e.g. Numba)

In [1]:
import numba
import numpy as np


from functools import wraps
from time import time, sleep
from itertools import permutations

#### Define a function-decorator for timing the runtime of your functions

Below is some code that you can use to wrap your functions so that you can time them individually.  The function defined imeadiately below the *timer()* is an example of how to use the warpper. In the cell that follows, execute this example function. (Note, for this timer, were are making use of something called *decorators*, but a discussion about this feature is outside the scope of this class.)


In [2]:
# defining a function decorator for timing other functions
def mytimer(func):
    @wraps(func)
    def wrap(*args, **kwargs):
        t_start = time() # Students look here
        result = func(*args, **kwargs)
        t_end = time() # Students also look here. This is how you can time things inside functions/classes
        print(f'Function: \'{func.__name__}({kwargs})\' executed in {(t_end-t_start):6.5f}s\n')
        return result, t_end-t_start
    return wrap
    

# Example of how to use. NOTE the "@mytimer" stated just above the function definition
@mytimer
def example_sum_timer_wrap(N):
    """ Sum the squares of the intergers, i, contained in [1-N] """
    return np.sum(np.arange(1,N+1)**2)

#### Run the example function

Run the example function for each of the following instances of $N = 10^5, 10^6, 10^7, 10^8$. Please examine the results, in particular, how the runtime changes with respect to $N$.


**Answer** the following questions in the markdown cell that follow the code. 
* For each factor-of-ten increase in $N$, roughly how much longer was the runtime of the function?
* Does this slowdown in the runtime make sense? Why or why not?

In [10]:
# Call the example function above for each value of N (try making one-call first, then loop)
exe_time = []
for n in [1e5, 1e6,1e7,1e8]:
    _,t = example_sum_timer_wrap(n)
    exe_time.append(t)


Function: 'example_sum_timer_wrap({})' executed in 0.00064s

Function: 'example_sum_timer_wrap({})' executed in 0.00265s

Function: 'example_sum_timer_wrap({})' executed in 0.03430s

Function: 'example_sum_timer_wrap({})' executed in 0.60437s



In [11]:
for i in np.arange(0,len(exe_time)-1):
    slow_down = exe_time[i+1] / exe_time[i]
    print("The slowdowns are: " + str(np.around(slow_down,2)))

The slowdowns are: 4.16
The slowdowns are: 12.96
The slowdowns are: 17.62


#### Your answer for Exerciese 0 questions here:



- The slow downs from each 10-fold increase is shown above.
- I tried to run the above two cells several times, because I noticed the slowdowns do vary from run to run.
   - The first slowdown, i.e. from input sizes 1e5 to 1e6, roughly have a value of about 1-5.
   - The second slowdown varies from about 9 to 13 from run to run.
   - The third slowdown is smaller than the second slowdown, for the first run, but for subsequent runs, this number is around 15-20, i.e. slightly larger than the second slowdown.  

I don't think this pattern follows any specific big-O time complexity (I think a naive sum of squares should scale as O(N), clearly this is not the case here). However, a quick Google search said that a big-O type analysis is in general not suitable for Numpy code, as different portions of any Numpy function often has mixed big-O time complexity and the overall behavior is often complex. 

<br><br>
### Exercise 1: Naive matrix multiplication: 

For this exercise, we will write our own matrx-matrix multiplication function.  We are goint to be naieve about it, so we want to loop over individual indices, instead of using slicing (in the next section, we will parallelize this code and we want to see how well Numba does at speeding up the naive code).

#### Tasks for this exercise

1. Write a function with that calculates matrix-matrix multiplication such that $C = A\cdot B$, where $A$, $B$, and $C$ are 2D numpy arrays. Make the dtype for the $C$ matrix the same as the dtype for $A$. Below is a stencil you can start with. test your results for accuracy and performance against the np.dot() function. To time the np.dot() function, you can wrap it in another function and use the *mytimer* wrapper; however, please opy the code for testing the A and B dimensions into your function wrapper for the np.dot() function.  This keeps the runtime comparison as a more "apples-to-apples" like comparison.

```python
@mytimer
def my_naive_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in range(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
<br>

2. Define your functions in the cell below.
3. Test and compare the accuracy and runtime of your functions in the next cell.
    - Test with non-square Matrices: $A \in \mathbb{R}^{N\times K}$ and $B \in \mathbb{R}^{K \times M}$. With $N = 64$, $K = 32$, and $M = 128$.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 64, 128,$ and $256$.
    - Test the following three cases:
        - Case-1. $A$ and $B$ **both** have dtype=np.**float32** (make sure that $C$ is also of dtype=np.**float32**)
        - Case-2. $A$ has dtype=np.**float64**, but $B$ has dtype=np.**float32**
        - Case-3. $A$ and $B$ **both** have dtype=np.**float64** (make sure that $C$ is also of dtype=np.**float64**)
    - For all three case above:
        - Calculate and show the error by computing the sum of the differnece between the $C$ matrices computed from numpy.dot() and your my_naive_matmul() functions. Assume that numpy.dot() is correct
        - Calculate and show the *speedup* that the faster function has versus the slower function.
        - Comment on the which of the three cases is fastest, and comment on what the speedup of the fastest case is and why it is the fastest case.
4. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
A = np.random.rand(N, K)
B = np.random.rand("for-you-to-figure-out")
```

<br>

#### Write function definitions for Exercise 1 below

In [13]:
# Define your functions for Exercise 1 here.
@mytimer
def my_naive_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    AShape = np.shape(A)
    BShape = np.shape(B)
    # A is M by N
    M = AShape[0]
    N = AShape[1]
    # B is N2 by Z
    N2 = BShape[0]
    Z = BShape[1]
    
    if (N != N2):
        raise Exception('The matrices do not have the appropriate dimensions for multiplication')
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((M,Z))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    for i in np.arange(M):
        for j in np.arange(Z):
            for k in np.arange(N):
                C[i,j] += A[i,k] * B[k,j]
    
    # 4
    # return the 2D numpy array for C
    # that has the same dtype as A
    return C.astype(A.dtype)

In [14]:
@mytimer
def np_dot(A:np.ndarray, B:np.ndarray):
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    AShape = np.shape(A)
    BShape = np.shape(B)
    # A is M by N
    M = AShape[0]
    N = AShape[1]
    # B is N2 by Z
    N2 = BShape[0]
    Z = BShape[1]
    
    if (N != N2):
        raise DimensionError('The matrices do not have the appropriate dimensions for multiplication')
    # 2 use np.dot to perform matrix multiplication
    C = np.dot(A,B)
    
    # 3 return C, casting in to the same type as A
    return C.astype(A.dtype)
    

<br>

#### Write the test codes for Exercise 1 in the cell below.

In [15]:
# Test your code here for runtime and accuracy
N = 64
K = 32
M = 128

A = np.random.rand(N,K)
B = np.random.rand(K,M)

# Case 1
print("Case 1")
A.astype(np.float32)
B.astype(np.float32)
C1,t1 = my_naive_matmul(A,B)
C2,t2 = np_dot(A,B)
if t1<t2:
    print("The speed up is "+str(np.around(t2/t1,2)) +"times")
else:
    print("The speed up is "+str(np.around(t1/t2,2)) +"times")
print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
print("")

# Case 2
print("Case 2")
A.astype(np.float64)
B.astype(np.float32)
C1,t1 = my_naive_matmul(A,B)
C2,t2 = np_dot(A,B)
if t1<t2:
    print("The speed up is "+str(np.around(t2/t1,2)) +"times")
else:
    print("The speed up is "+str(np.around(t1/t2,2)) +"times")
print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
print("")

# Case 3
print("Case 3")
A.astype(np.float64)
B.astype(np.float64)
C1,t1 = my_naive_matmul(A,B)
C2,t2 = np_dot(A,B)
if t1<t2:
    print("The speed up is "+str(np.around(t2/t1,2)) +"times")
else:
    print("The speed up is "+str(np.around(t1/t2,2)) +"times")
print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
print("")

Case 1
Function: 'my_naive_matmul({})' executed in 0.25093s

Function: 'np_dot({})' executed in 0.00412s

The speed up is 60.92times
The level of accuracy is 10 to the -13th power

Case 2
Function: 'my_naive_matmul({})' executed in 0.23000s

Function: 'np_dot({})' executed in 0.00024s

The speed up is 974.45times
The level of accuracy is 10 to the -13th power

Case 3
Function: 'my_naive_matmul({})' executed in 0.21975s

Function: 'np_dot({})' executed in 0.00017s

The speed up is 1276.62times
The level of accuracy is 10 to the -13th power



In [23]:
# Case 1
print("Case 1")
for N in [64, 128, 256]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float32)
    B.astype(np.float32)
    
    print("Testing for N= " + str(N))

    C1,t1 = my_naive_matmul(A,B)
    C2,t2 = np_dot(A,B)
    if t1<t2:
        print("The speed up is "+str(np.around(t2/t1,2)) +"times")
    else:
        print("The speed up is "+str(np.around(t1/t2,2)) +"times")
    print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
    print("---------------------------------------------------------------------------------")
    print("")

# Case 2
print("Case 2")
for N in [64, 128, 256]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float64)
    B.astype(np.float32)
    
    print("Testing for N= " + str(N))

    C1,t1 = my_naive_matmul(A,B)
    C2,t2 = np_dot(A,B)
    if t1<t2:
        print("The speed up is "+str(np.around(t2/t1,2)) +"times")
    else:
        print("The speed up is "+str(np.around(t1/t2,2)) +"times")
    print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
    print("---------------------------------------------------------------------------------")
    print("")

# Case 3
print("Case 3")
for N in [64, 128, 256]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float64)
    B.astype(np.float64)
    
    print("Testing for N= " + str(N))

    C1,t1 = my_naive_matmul(A,B)
    C2,t2 = np_dot(A,B)
    if t1<t2:
        print("The speed up is "+str(np.around(t2/t1,2)) +"times")
    else:
        print("The speed up is "+str(np.around(t1/t2,2)) +"times")
    print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
    print("---------------------------------------------------------------------------------")
    print("")

Case 1
Testing for N= 64
Function: 'my_naive_matmul({})' executed in 0.22508s

Function: 'np_dot({})' executed in 0.00015s

The speed up is 1520.2times
The level of accuracy is 10 to the -14th power
---------------------------------------------------------------------------------

Testing for N= 128
Function: 'my_naive_matmul({})' executed in 1.64478s

Function: 'np_dot({})' executed in 0.00030s

The speed up is 5466.49times
The level of accuracy is 10 to the -14th power
---------------------------------------------------------------------------------

Testing for N= 256
Function: 'my_naive_matmul({})' executed in 13.13790s

Function: 'np_dot({})' executed in 0.00047s

The speed up is 27900.95times
The level of accuracy is 10 to the -13th power
---------------------------------------------------------------------------------

Case 2
Testing for N= 64
Function: 'my_naive_matmul({})' executed in 0.22933s

Function: 'np_dot({})' executed in 0.00023s

The speed up is 1001.97times
The level

#### Discussion for Exercise 1:

For the non-square matriced, Case 3, when both matrices have data type 'float64', has the largest speed up. 

For the square matrices, for each case of dtype combinations, the speed up always increases as N increases, indicating that there is a rather big performance difference between 'my_naive_matmul' and 'np.dot', with the latter clearly much more optimized to handle larger inputs. The speed up is the largest for Case 3, N = 256. 

I think the reason for the above observations is that 'np.dot' is not only more optimized when the input size is large, it also performs much better, compared to 'my_naive_matmul', when the data type becomes more complex, i.e. when data type is 'float64', 'my_naive_matmul' takes a bigger hit while 'np.dot', being more optimized for this, is less affected 

<br><br>
### Exercise 2: Using Numba to speed up matrix multiplication: 

For this exercise, numba to speedup our matrx-matrix multiplication function below. However, there is an interesting twist to this exercise. We will write six versions of the naive function you wrote above. One function for each of the six possible permutations of three loops used to calculate the multplication (i.e. ijk, ikj, jik, etc.)

#### The tasks for this exercise:
1. Make six copies your code for the my_naive_matmul(), one copy for each of the possible permutations and define them in the cell bellow:
    - One of these function will have the same loop order as the my_naive_matmul() function. 
    - However name these functions: numba_mul_\<perm\>(), where \<perm\> should be replaced by the specific loop order of the function.
2. In the cell that follows your function definitions, test and compare the accuracy and runtime of your functions against the numpy.dot() function.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 128, 256,$ and $512$.
    - For each matrix set their dtype as: dtype=np.float64
    - Calculate and show the error between your functions and the numpy.dot() function. (Same as in Exercise 1.)
    - Calculate and show the *speedup* that the fastest function has versus the all the other functions.
    - You should notice that one of your permutation functions is faster than the others. For this case show the following:
        - Calculate and show the *speedup* that the fastest permutation function has versus the my_naive_matmul().
        - Calculate and show the *speedup* that the fastest permutation function has when $A$, $B$, and $C$ are all of dtype=np.float64 vs all are of dtype=np.float32.
3. Create your matrices using random numbers. (Same as in Exercise 1.) 
4. For each function, you need to add a function decorator for numba. Numba will *JIT* the function (**J**ust **I**n **T**ime compilation). 
    - Use the flag to keep a "cache" of the compiled code (not to be confused with CPU cache). **Note**: to get accurate timings, you will need to run your tests **twice**, because during the first run, the code is compiled and this compile time will be included in your runtime. After the first run, the compiled binary will be stored, so consecutive runs will be faster.
    - Use the flag to use *fast math*
    - Use the flag to disable the Python Global Interpretor Lock (GIL).
    - The code below should get you started, but it is incomplete.
    
```python   
    
@mytimer
@numba.njit(<flagname_caching>=<flag>, <flagname_fast_math>=<flag>, <flagname_no_gil>=<flag>)
def numba_mul_<perm>(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>): # !!!! LOOK HERE !!!!
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
    
6. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the questions asked in the markdown cell.


<br>

#### Write function definitions for Exercise 2 below

In [24]:
# Define your functions for Exercise 2 here.

############## ijk
@mytimer
@numba.njit(nogil=True, fastmath=True, cache=True)
def numba_mul_ijk(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    AShape = np.shape(A)
    BShape = np.shape(B)
    # A is M by N
    M = AShape[0]
    N = AShape[1]
    # B is N2 by Z
    N2 = BShape[0]
    Z = BShape[1]
    
    if (N != N2):
        raise Exception('The matrices do not have the appropriate dimensions for multiplication')
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((M,Z))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    for i in np.arange(M):
        for j in np.arange(Z):
            for k in np.arange(N):
                C[i,j] += A[i,k] * B[k,j]
    
    # 4
    # return the 2D numpy array for C
    # that has the same dtype as A
    return C.astype(A.dtype)

############## ikj
@mytimer
@numba.njit(nogil=True, fastmath=True, cache=True)
def numba_mul_ikj(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    AShape = np.shape(A)
    BShape = np.shape(B)
    # A is M by N
    M = AShape[0]
    N = AShape[1]
    # B is N2 by Z
    N2 = BShape[0]
    Z = BShape[1]
    
    if (N != N2):
        raise Exception('The matrices do not have the appropriate dimensions for multiplication')
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((M,Z))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    for i in np.arange(M):
        for k in np.arange(N):
            for j in np.arange(Z):            
                C[i,j] += A[i,k] * B[k,j]
    
    # 4
    # return the 2D numpy array for C
    # that has the same dtype as A
    return C.astype(A.dtype)

############## jik
@mytimer
@numba.njit(nogil=True, fastmath=True, cache=True)
def numba_mul_jik(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    AShape = np.shape(A)
    BShape = np.shape(B)
    # A is M by N
    M = AShape[0]
    N = AShape[1]
    # B is N2 by Z
    N2 = BShape[0]
    Z = BShape[1]
    
    if (N != N2):
        raise Exception('The matrices do not have the appropriate dimensions for multiplication')
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((M,Z))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    for j in np.arange(Z):
        for i in np.arange(M):
            for k in np.arange(N):                 
                C[i,j] += A[i,k] * B[k,j]
    
    # 4
    # return the 2D numpy array for C
    # that has the same dtype as A
    return C.astype(A.dtype)

############## jki
@mytimer
@numba.njit(nogil=True, fastmath=True, cache=True)
def numba_mul_jki(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    AShape = np.shape(A)
    BShape = np.shape(B)
    # A is M by N
    M = AShape[0]
    N = AShape[1]
    # B is N2 by Z
    N2 = BShape[0]
    Z = BShape[1]
    
    if (N != N2):
        raise Exception('The matrices do not have the appropriate dimensions for multiplication')
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((M,Z))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    for j in np.arange(Z):
        for k in np.arange(N): 
            for i in np.arange(M):                        
                C[i,j] += A[i,k] * B[k,j]
    
    # 4
    # return the 2D numpy array for C
    # that has the same dtype as A
    return C.astype(A.dtype)

############## kij
@mytimer
@numba.njit(nogil=True, fastmath=True, cache=True)
def numba_mul_kij(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    AShape = np.shape(A)
    BShape = np.shape(B)
    # A is M by N
    M = AShape[0]
    N = AShape[1]
    # B is N2 by Z
    N2 = BShape[0]
    Z = BShape[1]
    
    if (N != N2):
        raise Exception('The matrices do not have the appropriate dimensions for multiplication')
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((M,Z))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication  
    for k in np.arange(N): 
        for i in np.arange(M): 
            for j in np.arange(Z):
                C[i,j] += A[i,k] * B[k,j]
    
    # 4
    # return the 2D numpy array for C
    # that has the same dtype as A
    return C.astype(A.dtype)

############## kji
@mytimer
@numba.njit(nogil=True, fastmath=True, cache=True)
def numba_mul_kji(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    AShape = np.shape(A)
    BShape = np.shape(B)
    # A is M by N
    M = AShape[0]
    N = AShape[1]
    # B is N2 by Z
    N2 = BShape[0]
    Z = BShape[1]
    
    if (N != N2):
        raise Exception('The matrices do not have the appropriate dimensions for multiplication')
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    C = np.zeros((M,Z))
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication  
    for k in np.arange(N): 
        for j in np.arange(Z):
            for i in np.arange(M):         
                C[i,j] += A[i,k] * B[k,j]
    
    # 4
    # return the 2D numpy array for C
    # that has the same dtype as A
    return C.astype(A.dtype)

<br>

#### Write the test codes for Exercise 2 in the cell below.

In [25]:
# Test your code here for runtimes, accuracy, and speedups
print("Case ijk")
for N in [128, 256, 512]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float64)
    B.astype(np.float64)
    
    print("Testing for N= " + str(N))

    C1,t1 = numba_mul_ijk(A,B)
    C2,t2 = np_dot(A,B)
    if t1<t2:
        print("The speed up is "+str(np.around(t2/t1,2)) +"times. Numba is faster.")
    else:
        print("The speed up is "+str(np.around(t1/t2,2)) +"times. np.dot is faster.")
    if abs(np.sum(C1-C2)) != 0:
        print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
    else:
        print("The accuracy is within machine error")
    print("---------------------------------------------------------------------------------")
    print("")

print("Case ikj")
for N in [128, 256, 512]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float64)
    B.astype(np.float64)
    
    print("Testing for N= " + str(N))

    C1,t1 = numba_mul_ikj(A,B)
    C2,t2 = np_dot(A,B)
    if t1<t2:
        print("The speed up is "+str(np.around(t2/t1,2)) +"times. Numba is faster.")
    else:
        print("The speed up is "+str(np.around(t1/t2,2)) +"times. np.dot is faster.")
    if abs(np.sum(C1-C2)) != 0:
        print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
    else:
        print("The accuracy is within machine error")
    print("---------------------------------------------------------------------------------")
    print("")
    
print("Case jik")
for N in [128, 256, 512]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float64)
    B.astype(np.float64)
    
    print("Testing for N= " + str(N))

    C1,t1 = numba_mul_jik(A,B)
    C2,t2 = np_dot(A,B)
    if t1<t2:
        print("The speed up is "+str(np.around(t2/t1,2)) +"times. Numba is faster.")
    else:
        print("The speed up is "+str(np.around(t1/t2,2)) +"times. np.dot is faster.")
    if abs(np.sum(C1-C2)) != 0:
        print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
    else:
        print("The accuracy is within machine error")
    print("---------------------------------------------------------------------------------")
    print("")

print("Case jki")
for N in [128, 256, 512]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float64)
    B.astype(np.float64)
    
    print("Testing for N= " + str(N))

    C1,t1 = numba_mul_jki(A,B)
    C2,t2 = np_dot(A,B)
    if t1<t2:
        print("The speed up is "+str(np.around(t2/t1,2)) +"times. Numba is faster.")
    else:
        print("The speed up is "+str(np.around(t1/t2,2)) +"times. np.dot is faster.")
    if abs(np.sum(C1-C2)) != 0:
        print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
    else:
        print("The accuracy is within machine error")
    print("---------------------------------------------------------------------------------")
    print("")
    
print("Case kij")
for N in [128, 256, 512]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float64)
    B.astype(np.float64)
    
    print("Testing for N= " + str(N))

    C1,t1 = numba_mul_kij(A,B)
    C2,t2 = np_dot(A,B)
    if t1<t2:
        print("The speed up is "+str(np.around(t2/t1,2)) +"times. Numba is faster.")
    else:
        print("The speed up is "+str(np.around(t1/t2,2)) +"times. np.dot is faster.")
    if abs(np.sum(C1-C2)) != 0:
        print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
    else:
        print("The accuracy is within machine error")
    print("---------------------------------------------------------------------------------")
    print("")
    
print("Case kji")
for N in [128, 256, 512]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float64)
    B.astype(np.float64)
    
    print("Testing for N= " + str(N))

    C1,t1 = numba_mul_kji(A,B)
    C2,t2 = np_dot(A,B)
    if t1<t2:
        print("The speed up is "+str(np.around(t2/t1,2)) +"times. Numba is faster.")
    else:
        print("The speed up is "+str(np.around(t1/t2,2)) +"times. np.dot is faster.")
    if abs(np.sum(C1-C2)) != 0:
        print("The level of accuracy is 10 to the " +str(int(np.floor(np.log10(abs(np.sum(C1-C2))))))+"th power")
    else:
        print("The accuracy is within machine error")
    print("---------------------------------------------------------------------------------")
    print("")

Case ijk
Testing for N= 128
Function: 'numba_mul_ijk({})' executed in 0.01398s

Function: 'np_dot({})' executed in 0.00016s

The speed up is 86.25times. np.dot is faster.
The accuracy is within machine error
---------------------------------------------------------------------------------

Testing for N= 256
Function: 'numba_mul_ijk({})' executed in 0.05968s

Function: 'np_dot({})' executed in 0.00052s

The speed up is 114.3times. np.dot is faster.
The level of accuracy is 10 to the -12th power
---------------------------------------------------------------------------------

Testing for N= 512
Function: 'numba_mul_ijk({})' executed in 0.32544s

Function: 'np_dot({})' executed in 0.00307s

The speed up is 106.11times. np.dot is faster.
The level of accuracy is 10 to the -11th power
---------------------------------------------------------------------------------

Case ikj
Testing for N= 128
Function: 'numba_mul_ikj({})' executed in 0.01036s

Function: 'np_dot({})' executed in 0.00009s


In [27]:
# kij seems the fastest
for N in [128,256,512]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float64)
    B.astype(np.float64)
    
    print("Testing for N= " + str(N))
    
    tl = []                               # list of run times for each N
    
    tl.append(numba_mul_kij(A,B)[1])      # the fastest
    tl.append(my_naive_matmul(A,B)[1])    # without numba
    tl.append(np_dot(A,B)[1])             # np.dot
    tl.append(numba_mul_ijk(A,B)[1])
    tl.append(numba_mul_ikj(A,B)[1])    
    tl.append(numba_mul_jik(A,B)[1])
    tl.append(numba_mul_jki(A,B)[1])
    tl.append(numba_mul_kji(A,B)[1])
    
    print("For the case N =", str(N)+" The speed up of the fastest permutation, kij,"+\
          "over my_naive_matmul, np.dot, ijk,jik,jki,kij and kji are: ")
    print([str(np.around(tl[s]/tl[0],2))+"times" for s in range(1,len(tl))])
    print("---------------------------------------------------------------------------------")
    print("")

Testing for N= 128
Function: 'numba_mul_kij({})' executed in 0.00649s

Function: 'my_naive_matmul({})' executed in 1.69564s

Function: 'np_dot({})' executed in 0.00034s

Function: 'numba_mul_ijk({})' executed in 0.00468s

Function: 'numba_mul_ikj({})' executed in 0.00450s

Function: 'numba_mul_jik({})' executed in 0.00530s

Function: 'numba_mul_jki({})' executed in 0.00662s

Function: 'numba_mul_kji({})' executed in 0.00707s

For the case N = 128 The speed up of the fastest permutation, kij,over my_naive_matmul, np.dot, ijk,jik,jki,kij and kji are: 
['261.11times', '0.05times', '0.72times', '0.69times', '0.82times', '1.02times', '1.09times']
---------------------------------------------------------------------------------

Testing for N= 256
Function: 'numba_mul_kij({})' executed in 0.03087s

Function: 'my_naive_matmul({})' executed in 12.99686s

Function: 'np_dot({})' executed in 0.00044s

Function: 'numba_mul_ijk({})' executed in 0.04318s

Function: 'numba_mul_ikj({})' executed in 0.

In [28]:
# kij seems the fastest
for N in [128,256,512]:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    
    A.astype(np.float32)
    B.astype(np.float32)
    
    print("Testing for N= " + str(N))
    
    tl = []                               # list of run times for each N
    
    tl.append(numba_mul_kij(A,B)[1])      # the fastest
    tl.append(my_naive_matmul(A,B)[1])    # without numba
    tl.append(np_dot(A,B)[1])             # np.dot
    tl.append(numba_mul_ijk(A,B)[1])
    tl.append(numba_mul_ikj(A,B)[1])    
    tl.append(numba_mul_jik(A,B)[1])
    tl.append(numba_mul_jki(A,B)[1])
    tl.append(numba_mul_kji(A,B)[1])
    
    print("For the case N =", str(N)+" The speed up of the fastest permutation, kij,"+\
          "over my_naive_matmul, np.dot, ijk,jik,jki,kij and kji are: ")
    print([str(np.around(tl[s]/tl[0],2))+"times" for s in range(1,len(tl))])
    print("---------------------------------------------------------------------------------")
    print("")

Testing for N= 128
Function: 'numba_mul_kij({})' executed in 0.00771s

Function: 'my_naive_matmul({})' executed in 1.70651s

Function: 'np_dot({})' executed in 0.00024s

Function: 'numba_mul_ijk({})' executed in 0.00520s

Function: 'numba_mul_ikj({})' executed in 0.00439s

Function: 'numba_mul_jik({})' executed in 0.00563s

Function: 'numba_mul_jki({})' executed in 0.00884s

Function: 'numba_mul_kji({})' executed in 0.00688s

For the case N = 128 The speed up of the fastest permutation, kij,over my_naive_matmul, np.dot, ijk,jik,jki,kij and kji are: 
['221.34times', '0.03times', '0.67times', '0.57times', '0.73times', '1.15times', '0.89times']
---------------------------------------------------------------------------------

Testing for N= 256
Function: 'numba_mul_kij({})' executed in 0.03242s

Function: 'my_naive_matmul({})' executed in 13.09766s

Function: 'np_dot({})' executed in 0.00057s

Function: 'numba_mul_ijk({})' executed in 0.04475s

Function: 'numba_mul_ikj({})' executed in 0.

<br>

### Discussion for Exercise 2
1. What do you think is causing the differences in performance between the various permutations?
1. Which function is fastest (permutations or numpy.dot)? Why do you think this function is the fastest, and could there be multiple factors involved regarding the superior performance?
1. When comparing the matrix-matrix performance between the cases were all matrices had dtype=np.float64 vs.  all matrices having dtype=np.float32, which dtype was fastest? Roughtly what was the speedup when using this dtype vs the other?
1. Does Amdahl's Law play a major factor in the performance differences? Which part of the matrix-matrix multiplication was not parallelized (serial portion)? (Hint: which matricies did we reuse for each function?) Could we parallelize this part, and if so, are there caveats?
1. Do you think all codes should be parallelized? What about matrix-matrix multiplication? (This is a subjective question, but I am looking for a brief, but rational and informed arguement).
1. For those taking the class for **4 units**: Regarding the performance differences, which Law do you think is more relevant when comparing the performance differences between each of the functions in this exercise, Amdahl's Law or Gustafson's Law? Explain your reasoning. 

My answers:
1. This has to do with cache hits or miss. Numpy defaults to row-major, which means when a value is read in from memory, its neighbouring values from the same row are also being moved to the cache. Here, we have three matrices, A, B and C to access. The permutation that has the access pattern with the smallest rate of cahce miss will generally be faster.

2. np.dot() is always much faster, even though I have made sure to run the JIT compiled functions twice or more. I think the reason here is that Numba only provides significant performance gains as compared to native Python code, as it does here, but when compared with the highly-optimized Numpy code (and this optimization can really come from several aspects, e.g. Numpy achieves better vectorization and/or somehow minimizes cache misses etc.) it often falls behind. 

3. float32 is slightly faster. Roughly, it is about 1-5 % faster, although there is a variation from run to run and case to case. 
4. Yes. The serial part of matrix-matrix multiplication should be computing each individual element, $C_{ij}$, of the resulting matrix, which is done by performing dot-product of each row of the first matrix and each column of the second matrix. Because all of the individual multiplications in the dot product have to be performed *before* the sum is performed to get $C_{ij}$, this is the part that is limiting performance according to Amdahl's Law. This whole process though, can be parallelized and being done by different threads/core. As a result, as the matrix size increases, the portion of the serial tasks increases, leading to smaller performance gains for the 'numba_kij' as compared with 'my_naive_matmul'. Using the data type of 'float32' decreases the complexity for the serial part, hence making the performance better. I don't think this part can be parallelized in general, unless one uses some special techniques such as to reduce the matrix size to reduce the serial portion. But then at the end the individual blocks still have to be recombined, and one needs to carefully think about whether there are any gains for utilizing such special tricks.

5. Due to the nature of the tasks, certain codes are inherently not parallelizeable, so of course not all codes should be parallelized. As for the specific case of matrix-matrix multiplication, I think this is an area of active research and people are constantly trying to come up with novel techniques (the latest one I heard I think was last year, with an AI based method). While these new techniques often bring performance increase, I think they also tend to become less and less general, i.e. you need specific kinds of matrices or hardware to achieve that performance gain. 